# Earthquakes and hydraulic fracture in Oklahoma

Harvard CS109 final project, submitted December 2015

Kamrine Poels, Eric Fredickson, Thibaut Perol

## Notebook content

0. Introduction

1. Oklahoma state seismicity

2. Local seismicity

    2.1 Oklahoma city
    2.2 Prague
    2.3

# Table of Contents

* [0. Introduction](#0-Introduction)
  *   [0.1 Our data](#0.1-data)
  *   [0.2 Visualization of the earthquakes](#0.3-visualization)
  *  [0.3 Increase of the number of earthquake events in Oklahoma](#0.3-number-of-events)

* [1. Oklahoma State seismicity](#1-Oklahoma-state)
  * [1.1 Is the increase of earthquakes related to the increase of disposal wells](#1.1-increase-earthquakes-disposal-wells)

* [2. Local seismicity](#2.-Local-seismicity)
  *   [2.1 Oklahoma city](#2.1-Oklahoma-city) 
  *   [2.2 Prague](#2.2-Prague)



# 0. Introduction

Over the past years, Oklahoma has been impacted by a significant increase of earthquake events. The exponential increase of the number of earhquakes affects all range of magnitude, from the smallest to the largest earthquake sizes. Furthemore, in this state, the number of saltwater disposal wells have increased dramatically. The saltwater disposal comes from 'produce' water that was used to hydraulically fracturing a low permeability rock rich in unconventional oil. Once the oil is extracted, along with this saltwater, this latter is reinjected into deeper sedimentary formations. Sometimes, this formation is hydraulically connected to the crystalline bedrock and could potentially trigger earthquakes on preexisting faults. 

# 0.1 Our data

# 0.2 Visualization of the earthquakes

# 2. Local seismicity

# 2.1 Oklahoma city

# 2.2 Prague